In [45]:
!pip install selenium
!pip install undetected-chromedriver
import nltk
from nltk.util import ngrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np
import re
import requests
import html
import undetected_chromedriver as uc
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [20]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mettle\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mettle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [75]:
response = requests.get("https://api.pullpush.io/reddit/search/comment/?q=MSFT&subreddit=wallstreetbets")

if response.status_code == 200:
    data = response.json()
    body_html_list = [comment['body_html'] for comment in data['data'] if 'body_html' in comment]
else:
    print(f"Request failed with status code: {response.status_code}")

In [76]:
# Clean up data : 
def remove_html_tags(text):
    text = html.unescape(text)
    text = re.sub(r'<.*?>', '', text)
    text = text.replace('\n', ' ')
    return text.strip()

def preprocess_data(body_html_list):
    cleaned_comments = [remove_html_tags(html) for html in body_html_list]
    return cleaned_comments

    
comments = preprocess_data(body_html_list)

In [77]:
custom_words = {
    "tendies": 2.0,
    "diamond hands": 1.5,
    "moon": 2.0,
    "rocket": 1.8,
    "gainz": 1.5,
    "bullish": 1.2,
    "yolo": 0.5,
    "green": 1.3,
    "mom":-1,
    "mommy":-1,
    "printer go brrr": 1.0,
    "money printer": 2.0,
    "bagholder": -2.0,
    "paper hands": -1.5,
    "red": -1.2,
    "rekt": -1.8,
    "bag": -1.5,
    "short": -1.0,
    "crash": -2.0,
    "dump": -1.7,
    "bleeding": -2.0,
    "hold the line": 0.5,
    "degenerate": 0.0,
    "fomo": -0.5,
    "stonks": 1.0,
    "brrr":2,
    "fries":-1.0,
    "burger":-1.0,
    "gamble":-1.0,
    "f": -2.0
}

def generate_ngrams(text, n_values=[1, 2]):
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    ngram_list = []
    for n in n_values:
        ngram_list.extend([' '.join(grams) for grams in ngrams(tokens, n)])
    return ngram_list

def commentSentiment(comments, custom_words=None):
    sia = SIA()

    if custom_words:
        sia.lexicon.update(custom_words)

    results = []

    for line in comments:
        ngram_list = generate_ngrams(line)
        
        score = {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}
        
        for ngram in ngram_list:
            if ngram in sia.lexicon:
                ngram_score = sia.polarity_scores(ngram)
                score['neg'] += ngram_score['neg']
                score['neu'] += ngram_score['neu']
                score['pos'] += ngram_score['pos']
                score['compound'] += ngram_score['compound']
        
        score['comment'] = line
        results.append(score)

    df = pd.DataFrame.from_records(results)
    
    df['label'] = 0
    df.loc[df['compound'] > 0.10, 'label'] = 1
    df.loc[df['compound'] < -0.10, 'label'] = -1

    averageScore = df['label'].mean()

    return averageScore

print(commentSentiment(comments))

0.26


## Analzying for investor sentiment

In [105]:

# Set up Chrome options with a User-Agent header
#chrome_options = Options()
#chrome_options.add_argument("--headless")
#chrome_options.add_argument("--no-sandbox")
#chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36")

driver = uc.Chrome(headless=True,use_subprocess=False)

try:
    driver.get(url)
    
    div_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "markets-tipranks"))
    )
    
    iframe = div_element.find_element(By.TAG_NAME, "iframe")
    
    driver.switch_to.frame(iframe)

    target_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    extracted_text = target_element.text
    print("Extracted Text:", extracted_text)
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

Extracted Text: widgets.tipranks.com | Access denied (403)
Current session has been terminated.
For further information, do not hesitate to contact us.
Ref: 70.52.7.173 2024-11-03T00:45:09.633Z


In [108]:
import requests

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'priority': 'u=1, i',
    'referer': 'https://widgets.tipranks.com/content/v2/cnn/smartscoresmall/index.html?ticker=AAPL',
    'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
}

params = {
    'tickers': 'AAPL',
}

response = requests.get('https://widgets.tipranks.com/api/widgets/stockAnalysisOverview/', params=params, headers=headers)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")

[{'ticker': 'AAPL', 'hedgeFundTrendValue': -386045483.0, 'smartScore': 7, 'bloggerSectorAvg': 0.6914, 'bloggerBullishSentiment': 0.74, 'insidersLast3MonthsSum': -22602362.0, 'newsSentimentsBearishPercent': 0.1, 'newsSentimentsBullishPercent': 0.9, 'investorHoldingChangeLast7Days': -0.00302717579862374, 'investorHoldingChangeLast30Days': 0.00580673178394926, 'priceTarget': 245.84, 'convertedPriceTarget': 245.84, 'convertedPriceTargetCurrencyId': 1, 'fundamentalsReturnOnEquity': 1.574125, 'fundamentalsAssetGrowth': 0.03516, 'technicalsTwelveMonthsMomentum': 0.2878, 'sma': 'Positive', 'analystConsensus': 'ModerateBuy', 'hedgeFundTrend': 'Decreased', 'insiderTrend': 'SoldShares', 'investorSentiment': 'Neutral', 'newsSentiment': 'VeryBullish', 'bloggerConsensus': 'Bullish', 'marketCountryId': 1, 'isomic': 'XNAS'}]
